<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/vector_stores/WeaviateIndex_metadata_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weaviate Vector Store Metadata Filter

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index weaviate-client

#### Creating a Weaviate Client

In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-<your key here>"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import weaviate

# cloud
resource_owner_config = weaviate.AuthClientPassword(
    username="",
    password="",
)
client = weaviate.Client(
    "https://test.weaviate.network",
    auth_client_secret=resource_owner_config,
)

# local
# client = weaviate.Client("http://localhost:8081")

#### Load documents, build the VectorStoreIndex

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import WeaviateVectorStore
from IPython.display import Markdown, display

## Metadata Filtering

Let's insert a dummy document, and try to filter so that only that document is returned.

In [ ]:
from llama_index.schema import TextNode

nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
            "year": 1994,
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
            "year": 1972,
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
            "theme": "Fiction",
            "year": 2010,
        },
    ),
    TextNode(
        text="To Kill a Mockingbird",
        metadata={
            "author": "Harper Lee",
            "theme": "Mafia",
            "year": 1960,
        },
    ),
    TextNode(
        text="1984",
        metadata={
            "author": "George Orwell",
            "theme": "Totalitarianism",
            "year": 1949,
        },
    ),
    TextNode(
        text="The Great Gatsby",
        metadata={
            "author": "F. Scott Fitzgerald",
            "theme": "The American Dream",
            "year": 1925,
        },
    ),
    TextNode(
        text="Harry Potter and the Sorcerer's Stone",
        metadata={
            "author": "J.K. Rowling",
            "theme": "Fiction",
            "year": 1997,
        },
    ),
]

In [ ]:
from llama_index.storage.storage_context import StorageContext

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="LlamaIndex_filter"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes, storage_context=storage_context)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
retriever = index.as_retriever()
retriever.retrieve("What is inception?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='7c4ebf8f-da25-4143-bef3-22545c65949d', embedding=[0.0031094016, -0.024671212, -0.022274217, -0.036464944, -0.007159114, 0.011723607, -0.04006044, -0.027565459, -0.015758976, 0.007401364, 0.027845958, 0.029962454, 0.018767972, -0.004405118, 0.004127806, 0.0044274307, 0.027667457, -0.0076053636, -0.003036089, -0.012947605, -0.005702429, -0.01923972, 0.002084622, 0.009211861, 0.002052747, -0.012832856, 0.013629729, -0.029299455, 0.014993977, -0.01932897, 0.021993717, -0.009441361, -0.022197716, -0.009281985, -0.009409485, -0.0042999308, -0.011978607, -0.027233958, 0.015886476, -0.008026113, 0.014343728, 0.004079994, -0.006486553, -0.017072223, 0.0039429315, -0.014993977, -0.008312987, -0.021266967, -0.022847965, 0.0105123585, 0.016766224, 0.05278492, -0.042151436, -0.02621396, -0.0015650601, -0.0044338056, -0.007248364, 0.0059765535, 0.035495944, -0.008427737, -0.0025914335, -0.0074842386, -0.011360233, 0.0044656806, 0.0011020765, -0.008147238, -0.0293504

In [ ]:
from llama_index.vector_stores.types import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)


filters = MetadataFilters(
    filters=[
        MetadataFilter(key="theme", operator=FilterOperator.EQ, value="Mafia"),
    ]
)

retriever = index.as_retriever(filters=filters)
retriever.retrieve("What is inception about?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='f63fa125-e2b3-43ec-9f43-e5e2bcdb7688', embedding=[-0.0017992554, -0.023927106, -0.012763222, -0.035462487, -0.0096715875, 0.025614595, -0.0005037285, 0.00077920506, -0.021824038, -0.027831001, 0.023939699, 0.018700922, 0.028435476, -0.0018008294, 0.006447725, 0.015376314, 0.029065136, -0.008059656, 0.010395697, -0.0004836581, 0.0102949515, 0.0069199707, -0.029719982, -0.00061431265, 0.0047161584, -0.0010271339, 0.0043352135, -0.026974663, 0.021484021, -0.017517159, 0.012013925, -0.024078224, -0.0065106913, 0.0020322297, 0.010093461, -0.003085337, 0.005758247, -0.010477553, 0.0009570842, 0.014633314, 0.014003653, 0.007461479, -0.008280038, -0.016849719, 0.0058306577, -0.0027594878, 0.022516666, -0.011724281, -0.01363845, 0.014570348, 0.007058496, 0.03148303, -0.014809619, -0.030173339, 0.020048395, 0.0094763925, 0.0050089504, -0.016320804, 0.0038850063, -0.017869769, 0.010804977, 0.0044359593, -0.025589408, 0.0125932135, -0.0018937044, -0.013588077, -0.

In [ ]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters


filters = MetadataFilters(
    filters=[
        MetadataFilter(key="theme", value="Mafia"),
        MetadataFilter(key="year", value=1972),
    ]
)

retriever = index.as_retriever(filters=filters)
retriever.retrieve("What is inception?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='f63fa125-e2b3-43ec-9f43-e5e2bcdb7688', embedding=[-0.0017992554, -0.023927106, -0.012763222, -0.035462487, -0.0096715875, 0.025614595, -0.0005037285, 0.00077920506, -0.021824038, -0.027831001, 0.023939699, 0.018700922, 0.028435476, -0.0018008294, 0.006447725, 0.015376314, 0.029065136, -0.008059656, 0.010395697, -0.0004836581, 0.0102949515, 0.0069199707, -0.029719982, -0.00061431265, 0.0047161584, -0.0010271339, 0.0043352135, -0.026974663, 0.021484021, -0.017517159, 0.012013925, -0.024078224, -0.0065106913, 0.0020322297, 0.010093461, -0.003085337, 0.005758247, -0.010477553, 0.0009570842, 0.014633314, 0.014003653, 0.007461479, -0.008280038, -0.016849719, 0.0058306577, -0.0027594878, 0.022516666, -0.011724281, -0.01363845, 0.014570348, 0.007058496, 0.03148303, -0.014809619, -0.030173339, 0.020048395, 0.0094763925, 0.0050089504, -0.016320804, 0.0038850063, -0.017869769, 0.010804977, 0.0044359593, -0.025589408, 0.0125932135, -0.0018937044, -0.013588077, -0.

In [ ]:
from llama_index.vector_stores.types import (
    FilterOperator,
    FilterCondition,
)


filters = MetadataFilters(
    filters=[
        MetadataFilter(key="theme", value="Fiction"),
        MetadataFilter(key="year", value=1997, operator=FilterOperator.GT),
    ],
    condition=FilterCondition.OR,
)

retriever = index.as_retriever(filters=filters)
retriever.retrieve("Harry Potter?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[NodeWithScore(node=TextNode(id_='9949cb24-4f5e-4532-9eac-eaaa0b368072', embedding=[0.013188333, -0.013766769, -0.039642125, -0.007294717, -0.010765331, 0.017173113, -0.02822766, -0.05074809, -0.021042205, -0.01610622, 0.025476879, 0.014345204, 0.01700601, 0.0012805923, -0.0103797065, -0.00085801294, 0.021916285, -0.011260214, 0.015424951, -0.014962203, -0.01744305, -0.009261398, -0.006909093, 0.009737001, -0.010116197, -0.0015617764, 0.022520429, -0.041415993, 0.020103853, 0.0037019884, 0.007866725, -0.00535053, -0.026453791, -0.023844404, -0.03432052, 0.008503005, -0.02691654, -0.009344949, 0.00577793, -0.0037951807, 0.03043857, 0.013818186, -0.01881844, -0.015052182, -0.01691603, -0.013856748, 0.011915775, -0.017198822, -0.030284321, 0.004849219, 0.017700132, 0.03252094, -0.028638992, -0.023600176, -0.004685329, 0.0008764907, -0.0014115437, 0.006462412, 0.0075132367, -0.016556114, -0.0100455, -0.0031637219, -0.0058711222, -0.015090744, -0.00502275, -0.011498016, -0.020733705, -0.007